# Environmetn Setup

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
!pip install -qU "langchain[openai]" # select chat model OpenAI
!pip install -U langchain langchain-core langchain-community

In [2]:
!pip install -qU langchain-openai # select embeddings model OpenAI
!pip install -qU langchain-community # select vector store FAISS
!pip install jq
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 53.0 MB/s eta 0:00:00


In [3]:
!pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.1 MB/s eta 0:0

In [4]:
from openai import OpenAI
import json
import faiss
import re
import getpass
import os

In [5]:

from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import HTMLSectionSplitter
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from langchain.chains import create_extraction_chain_pydantic
from langchain import hub
from langchain_core.pydantic_v1 import BaseModel
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from typing import Optional, List


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [7]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI: ··········


In [8]:
from google.colab import drive
drive.mount('/content/drive')

%cd  /content/drive/MyDrive/ECE1508_Project/Codes

Mounted at /content/drive
/content/drive/MyDrive/ECE1508_Project/Codes


# Helpers

## Load Test Doc

In [9]:
def metadata_func(example: dict, _: dict) -> dict:
    return {
        "question_text": example.get("question_text"),
        "Title": example.get("title", "Untitled")
    }

In [61]:
def metadata_func_2(example: dict, _: dict) -> dict:
    return {
        "question_text": example.get("question_text"),
        "gold_answer": example.get("gold_answer"),
        "Title": example.get("title", "Untitled")
    }

In [21]:
def load_documents(file_path):
  loader=JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="document_text",
    metadata_func=metadata_func
  )
  documents=loader.load()

  return documents

In [62]:
def load_documents_2(file_path):
  loader=JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="document_text",
    metadata_func=metadata_func_2
  )
  documents=loader.load()

  return documents

## Vector Store Helpers

In [12]:
def create_faiss_vec_store(elemnts_to_emb, folder_name):
  vectorstore=FAISS.from_documents(elemnts_to_emb,embedding=OpenAIEmbeddings())
  vectorstore.save_local(folder_name)
  return vectorstore

## Retriver Helpers

In [13]:
# retrieve the top K contents
def retrieve_section(in_retriever,query,top_k):
  results=in_retriever.get_relevant_documents(query)
  if not results:
    return None
  top_5_match=results[:top_k]

  return top_5_match


In [14]:
#Run retriver for the input query
def get_retrieve_section(in_retriever,in_query,top_k):
  print(f"Retrieving answer for query: {in_query}")
  relevant_sections=retrieve_section(in_retriever,in_query,top_k)
  return relevant_sections

# Baseline Chunking & Save to Vector


In [85]:
#Vector stpre folder path
#Recheck pwd
!ls
baseline_vector_folder = './Baseline_vec'

 Baseline.ipynb		        L1_vector		     Proposition_Light.ipynb
 evaluation		        L1_vector_test		     Proposition_Sample.ipynb
 Evaluation.ipynb	        L1_vector_test_2	     rag_sw_ver2.ipynb
 gold_test_file_30.json         L2_vector_prop		     test_single_doc.json
'L1_Process_Chunk&Save.ipynb'   Proposition_Complete.ipynb


In [19]:
file_path="gold_test_file_30.json"
test_documents = load_documents(file_path)
print(f"{len(test_documents)} Documents")

30 Documents


In [63]:
file_path="gold_test_file_30.json"
test_documents_gold = load_documents_2(file_path)
print(f"{len(test_documents)} Documents")

30 Documents


In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import shutil
import time

In [76]:
def baseline_chunk(documents):

  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True
  )
  all_splits = text_splitter.split_documents(documents)

  return all_splits

In [86]:
def base_process_document(doc):


  chunks = baseline_chunk(doc)
  print(f"Split document into {len(chunks)} sub-documents.")

  # Embed and save to Chroma DB
  if os.path.exists(baseline_vector_folder):
    shutil.rmtree(baseline_vector_folder)
    time.sleep(1)
  embeddings = OpenAIEmbeddings()
  base_vectorstore = Chroma.from_documents(documents=chunks,
                                      embedding=embeddings,
                                      persist_directory=baseline_vector_folder)

  print(f"{len(doc)} documents {len(chunks)} chunks sucessfully processed and saved to {baseline_vector_folder}")

  return base_vectorstore



In [87]:
base_vectorstore = base_process_document(test_documents)

Split document into 3023 sub-documents.
30 documents 3023 chunks sucessfully processed and saved to ./Baseline_vec


## Retrieve & Generate


In [88]:
embeddings = OpenAIEmbeddings()
base_vectorstore = Chroma(
    persist_directory=baseline_vector_folder,
    embedding_function=embeddings
)
#Verify local load result
total_docs = base_vectorstore._collection.count()
if total_docs > 0:
    print(f"Vectorstore contains {total_docs} documents")
else:
    print("Vectorstore is empty")


Vectorstore contains 3023 documents


In [89]:
top_k = 5
base_retriever=base_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":top_k})

In [90]:
def get_API_response(client,sys_prompt,user_prompt,temp,topp):
  completion=client.chat.completions.create(
      model="gpt-4o",
      temperature=temp,
      top_p=topp,
      messages=[
          {"role":"system","content":sys_prompt},
          {"role":"user","content":user_prompt}
      ],
  )
  response=completion.choices[0].message.content
  return response

In [91]:
def retrieve_page_content(query):
  results=base_retriever.get_relevant_documents(query)
  if not results:
    return None

  print(results)

  return results

In [71]:
def build_question_to_gold_answer_map(in_documents):

    question_to_gold = {}

    for doc in in_documents:
        question = doc.metadata.get("question_text", "").strip()
        gold = doc.metadata.get("gold_answer", {})
        question_to_gold[question] = gold

    return question_to_gold


In [103]:
def RAG_base(questions,question_to_gold_map,mode='test'):

  answer_NQ_client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
  temp=1.0
  topp=1.0

  run_results = []

  for i in range(len(questions)):
    question_curr = questions[i]
    print(f"============={i+1} Question:{question_curr}=============")

    relevant_content = retrieve_page_content(question_curr)

    sys_prompt=""
    user_prompt=f"""
    Answer the question based on the relevent contents. If you don't know the answer, say 'I don't have the answer'
    Question: {question_curr}
    Relevant contents:{relevant_content}
    """
    response=get_API_response(answer_NQ_client,sys_prompt,user_prompt,temp,topp)

    print('-------Final Answer:-------------')
    print(f"Question: {question_curr}\n Response: {response}")

    #save to output file
    gold_answer = question_to_gold_map.get(question_curr.strip(), "")
    run_results.append({
      "input_question": question_curr,
      "retrieved_contexts": [doc.page_content for doc in relevant_content],
      "response": response,
      "gold_answer": gold_answer
    })

    if mode == 'test':
      break
    else:
      continue

  return run_results


In [104]:
test_questions = [
    eachDoc.metadata["question_text"]
    for eachDoc in test_documents
    if "question_text" in eachDoc.metadata
]

print(test_questions)

['what episode does olivia die in the walking dead', 'actor playing krishna in mahabharat on star plus', 'is the game show the chase still on tv', 'when was figure skating introduced to the olympics', 'setting of the ones who walk away from omelas', 'who does the voice of angela on family guy', 'how many episodes of gossip girl is there', 'who plays amy on the secret life of an american teenager', 'where is the roman forum located in rome', 'where is archangel raphael mentioned in the bible', 'who plays chuck on the tv show chuck', "who won the women's world cup championship in 2017", 'what is the southern most point in canada', 'how many episodes will prison break season 5 have', 'what does c class stand for in mercedes benz', 'how many years did it take to build the colosseum in rome', 'who is the present governor of puerto rico', 'when does episode 131 come out for dragon ball super', "what's the hottest natural pepper in the world", 'list of submissions to the 87th academy awards f

In [107]:
question_to_gold_map = build_question_to_gold_answer_map(test_documents_gold)
baseline_run_results = RAG_base(test_questions,question_to_gold_map,mode='prod')

=============1 Question:what episode does olivia die in the walking dead=============
[Document(id='11ac1a13-94b8-4109-832e-93597e582e6c', metadata={'Title': 'Untitled', 'question_text': 'what episode does olivia die in the walking dead', 'start_index': 0}, page_content='Olivia ( the Walking Dead ) - wikipedia <H1> Olivia ( the Walking Dead ) </H1> <Table> <Tr> <Th_colspan="2"> Olivia </Th> </Tr> <Tr> <Td_colspan="2"> The Walking Dead character </Td> </Tr> <Tr> <Td_colspan="2"> Olivia , as portrayed by Ann Mahoney in the television series ( left ) and in the comic book series ( right ) . </Td> </Tr> <Tr> <Th> First appearance </Th> <Td> <Ul> <Li> Comic : </Li> <Li> `` Issue # 70 \'\' ( 2010 ) </Li> <Li> Television : </Li> <Li> `` Remember \'\' ( 2015 ) </Li> </Ul>'), Document(id='ac2c2969-e318-4663-8f48-35b56dfeb728', metadata={'Title': 'Untitled', 'question_text': 'what episode does olivia die in the walking dead', 'start_index': 4955}, page_content="to know who made the bullet for he

In [108]:
def save_test_output(run_results):
  output_path = "./evaluation/run_results_baseline_.json"

  with open(output_path, "w", encoding="utf-8") as f:
      json.dump(run_results, f, indent=4, ensure_ascii=False)

  print(f"Saved {len(run_results)} results to {output_path}")

In [110]:
save_test_output(baseline_run_results)

Saved 30 results to ./evaluation/run_results_baseline_.json
